# Data Cleaning Listado de Productores Autorizados

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

In [ ]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [ ]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    dataframes = []
    for file in csv_files:
        try:
            # Try to read the CSV file with cp1252 encoding
            df = pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1)
        except UnicodeDecodeError:
            # If it fails, try to read the CSV file with utf-8 encoding
            df = pd.read_csv(file, encoding='utf-8', index_col=0, skiprows=1)
        
        # Print the columns of the current DataFrame
        print(f"Columns in {file}: {df.columns.tolist()}")
        
        dataframes.append(df)

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, ignore_index=True)

    return merged_df


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

Lectura del dataset del INEGI

In [ ]:
path_dataset_inegi = '../../data/inegi/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

Lectura del listado de productores autorizados

In [ ]:
listado_productores = load_datasets('../../data/productores_autorizados')

# 2. Limpieza de los datos

## 2.1 INEGI

In [ ]:
# Revisamos las columnas del dataset
dataset_inegi.columns

In [ ]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

In [ ]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [ ]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

In [ ]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [ ]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

In [ ]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

In [ ]:
dataset_inegi_clean.info()

In [ ]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

In [ ]:
# Creamos una columna con la clave única por municipio

dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[
    'CVE_MUN'].astype(str)

dataset_inegi_clean.head()

In [ ]:
dataset_inegi_clean.tail()

### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [ ]:
# Estandarizamos la limpieza de los datos
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)

In [ ]:
dataset_inegi_clean

In [ ]:
dataset_inegi_clean.shape

## 2.2 Listado de Productores Autorizados

In [ ]:
listado_productores.columns

In [ ]:
listado_productores = listado_productores.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'])

In [ ]:
listado_productores.head()

In [ ]:
# Mostramos el tipo de dato de cada columna
listado_productores.dtypes
listado_productores.info()

In [ ]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
COLUMNS_TO_VIEW = ['ESTADO', 'MUNICIPIO', 'PAQUETE']
for column in COLUMNS_TO_VIEW:
    unique_values = listado_productores[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\n{column}\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)

In [ ]:

municipio_counts = listado_productores['MUNICIPIO'].value_counts().reset_index()
municipio_counts.columns = ['MUNICIPIO', 'COUNT']
print(municipio_counts)

¿Qué ESTADOS tienen algún municipio NULL?

In [ ]:
municipios_null = listado_productores[listado_productores['MUNICIPIO'].isnull()]

# ¿Qué ESTADOS tienen algún municipio Null?
print(municipios_null['ESTADO'].unique())

In [ ]:
# Filas duplicadas y filas únicas
duplicated_rows_productores = listado_productores.duplicated()
number_of_duplicated_rows_productores = duplicated_rows_productores.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows_productores}")
print(f"El número de filas únicas es: {listado_productores.shape[0] - number_of_duplicated_rows_productores}")

In [ ]:
duplicated_rows = listado_productores[listado_productores.duplicated()]

# Imprimir las filas duplicadas
print(duplicated_rows)

In [ ]:
listado_productores.drop_duplicates(inplace=True)

No hay filas completas duplicadas.

In [ ]:
# Valores nulos
listado_productores.isna().sum()

Existen 75512 filas con el municipio nulo. 
Y muchas filas con nombre y apellidos vacíos.


In [ ]:
# Analicemos dónde los municipios están vacíos
listado_productores[(listado_productores['MUNICIPIO'].isna())]


Todas las filas con municipios vacíos vienen del estado de Veracruz de Ignacio de la Llave.

In [ ]:
listado_productores.info()

### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los nombres de municipios en un dataset más ligero llamado **Estados_productores**.

In [ ]:
# Seleccionar solo las dos primeras columnas
Estados_productores = listado_productores[['ESTADO', 'MUNICIPIO']]

# Obtener las filas únicas
Estados_productores = Estados_productores.drop_duplicates()

In [ ]:
Estados_productores.shape

In [ ]:
# Estandarizamos la limpieza de los datos
Estados_productores['ESTADO_Clean'] = Estados_productores['ESTADO'].apply(clean_text)
Estados_productores['MUNICIPIO_Clean'] = Estados_productores['MUNICIPIO'].apply(clean_text)

Creamos columna clave

In [ ]:
# Primero creemos una columna clave en cada dataset -> Estados productores

Estados_productores["ESTADO_Clean"] = Estados_productores["ESTADO_Clean"].astype(str)
Estados_productores["MUNICIPIO_Clean"] = Estados_productores["MUNICIPIO_Clean"].astype(str)

Estados_productores["KEY_prod"] = Estados_productores["ESTADO_Clean"] + "-" + Estados_productores["MUNICIPIO_Clean"]

In [ ]:
Estados_productores.shape

# 3. Diccionario de los datasets de INEGI Y PRODUCTORES AUTORIZADOS

A este punto llegamos con 3 datasets:  
    1. dataset_inegi_clean: municipios y estados de INEGI con los nombres estandarizados  
    2. Productores_Autorizados: listado de productores autorizados con municipios y estados estandarizados  
    3. Estados_productores: Municipios y Estados únicos que aparecen en el listado de Productores Autorizados, sin el formato INEGI.  

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Estados_productores. Para ello haremos un Left join entre Estados_productores y dataset_inegi_clean.

### 3.1 Left join

In [ ]:
# Primero creemos una columna clave en cada dataset -> INEGI

dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY_inegi"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

In [ ]:
dataset_inegi_clean.info()

In [ ]:
Estados_productores.info()

In [ ]:
Estados_productores = Estados_productores.drop(['ESTADO', 'MUNICIPIO'], axis=1)
Estados_productores = Estados_productores.drop_duplicates()
Estados_productores.shape

Comprobamos que la columna KEY es realmente clave.

In [ ]:
Estados_productores['KEY_prod'].nunique()

In [ ]:

# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge(df_inegi, df_prod, key1, key2, threshold=96, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='inner',
                         suffixes=('_inegi', '_prod'))
    
    return df_merged


In [ ]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge(dataset_inegi_clean, Estados_productores, 'KEY_inegi', 'KEY_prod')
diccionario.drop_duplicates(subset=['KEY_inegi'], inplace=True)

# Mostrar el resultado
diccionario.columns

In [ ]:
diccionario.head()

In [ ]:
diccionario.to_csv('../../data/productores_autorizados/diccionarios_E1/diccionario_prod.csv', index=False)

## 3.2 Productores Autorizados

Esta sección se encarga de completar el listado original de Productores Autorizados con los nombre corregido de INEGI usando el diccionario.

In [ ]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_productores['ESTADO_Clean'] = listado_productores['ESTADO'].apply(clean_text)
listado_productores['MUNICIPIO_Clean'] = listado_productores['MUNICIPIO'].apply(clean_text)
listado_productores['Estado-mun-KEY'] = listado_productores['ESTADO_Clean'].astype(str) + '-' + listado_productores[
    'MUNICIPIO_Clean'].astype(str)

diccionario_Sin_VC = diccionario[diccionario["NOM_ENT"] != "Veracruz de Ignacio de la Llave"]

In [ ]:
listado_productores.shape

In [ ]:
diccionario_Sin_VC = diccionario_Sin_VC[['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'CVE_MUN_Unique',
       'KEY_inegi', 'best_match', 'match_score', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'KEY_prod']]

diccionario_Sin_VC.shape

In [ ]:
diccionario_Sin_VC.to_csv('../../data/productores_autorizados/diccionarios_E1/diccionario_prod_sin_VERACRUZ.csv', index=False)
diccionario_Sin_VC.columns

In [ ]:
# Lectura del diccionario manipulado
diccionario_manipulado = pd.read_csv('../../data/productores_autorizados/diccionarios_E1/diccionario_prod_sin_VERACRUZ.csv')

In [ ]:
# Hacer el join
listado_productores_complete = pd.merge(listado_productores, diccionario_manipulado, left_on="Estado-mun-KEY",
                                        right_on="KEY_prod", how='left', suffixes=('_prod', '_inegi'))

In [ ]:
listado_productores_complete[['CVE_ENT', 'CVE_MUN']] = listado_productores_complete['CVE_MUN_Unique'].str.split('-',
                                                                                                                expand=True)


In [ ]:
listado_productores_complete.columns

In [ ]:
# Seleccionamos las columnas que nos interesan
listado_productores_complete = listado_productores_complete[
    ['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_inegi', 'NOM_ENT', 'NOM_MUN', 'CVE_ENT', 'CVE_MUN']]

In [ ]:
# Revisamos el dataset
print(listado_productores_complete.shape)
print(listado_productores_complete.columns)
print(listado_productores_complete.head())

In [ ]:
listado_productores_complete.to_csv('../../data/listados_completos/listado_productores_complete2023.csv', index=False)

# Adicional

In [ ]:
unique_listado = listado_productores['Estado-mun-KEY'].unique()
unique_diccionario = diccionario_Sin_VC['KEY_prod'].unique()

# To get the common values
common_values = set(unique_listado).intersection(unique_diccionario)
print("Common values:", common_values)

# To get values in listado_productores not in diccionario_Sin_VC
in_listado_not_diccionario = set(unique_listado) - set(unique_diccionario)
print("Values in 'listado_productores' not in 'diccionario_Sin_VC':", in_listado_not_diccionario)

In [ ]:
# To get values in diccionario_Sin_VC not in listado_productores
in_diccionario_not_listado = set(unique_diccionario) - set(unique_listado)
print("Values in 'diccionario_Sin_VC' not in 'listado_productores':", in_diccionario_not_listado)
# Create a DataFrame with unique values from 'listado_productores'
df_relations = pd.DataFrame(unique_listado, columns=['Estado-mun-KEY'])# Check if each value is present in 'diccionario_Sin_VC'
df_relations['In_diccionario_Sin_VC'] = df_relations['Estado-mun-KEY'].isin(unique_diccionario)

In [ ]:
# Repeat the process for 'diccionario_Sin_VC'
df_relations_diccionario = pd.DataFrame(unique_diccionario, columns=['KEY_prod'])
df_relations_diccionario['In_listado_productores'] = df_relations_diccionario['KEY_prod'].isin(unique_listado)

# Merge the two DataFrames
df_relations = pd.merge(df_relations, df_relations_diccionario, left_on='Estado-mun-KEY', right_on='KEY_prod',
                        how='outer')

df_relations

In [ ]:
df_relations[(df_relations['In_diccionario_Sin_VC'] == False) | (df_relations['In_listado_productores'] == False)]